In [ ]:
# Import benötigter Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# Style konfigurieren
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Pfade definieren
DATA_DIR = Path("../data/processed")
MODEL_DIR = Path("../models")
PLOTS_DIR = Path("../plots")
PLOTS_DIR.mkdir(exist_ok=True)

print("✓ Bibliotheken erfolgreich importiert")

## 1. Model und Daten laden

In [ ]:
# Model laden
model_path = MODEL_DIR / "model_latest.joblib"

if not model_path.exists():
    print("❌ Kein trainiertes Model gefunden!")
    print(f"   Bitte zuerst 'python train.py' ausführen")
else:
    model = joblib.load(model_path)
    print(f"✓ Model geladen: {model_path}")
    print(f"  Model Type: {type(model).__name__}")

In [ ]:
# Daten laden
features_path = DATA_DIR / "features.parquet"
target_path = DATA_DIR / "target.parquet"

if not features_path.exists() or not target_path.exists():
    print("❌ Keine prozessierten Daten gefunden!")
    print(f"   Bitte zuerst 'python train.py' ausführen")
else:
    X = pd.read_parquet(features_path)
    y = pd.read_parquet(target_path)['service_time_in_minutes']
    
    print(f"✓ Daten geladen")
    print(f"  Features Shape: {X.shape}")
    print(f"  Target Shape: {y.shape}")

## 2. Predictions generieren

In [ ]:
# Predictions
y_pred = model.predict(X)

# Metriken berechnen
rmse = np.sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("=" * 60)
print("MODEL PERFORMANCE METRIKEN")
print("=" * 60)
print(f"\nRoot Mean Squared Error (RMSE): {rmse:.4f} Minuten")
print(f"Mean Absolute Error (MAE):      {mae:.4f} Minuten")
print(f"R² Score:                        {r2:.4f}")
print(f"\nInterpretation:")
print(f"  - Im Durchschnitt weicht die Vorhersage um {mae:.2f} Minuten ab")
print(f"  - Das Model erklärt {r2*100:.2f}% der Varianz in den Daten")

## 3. Visualisierung 1: Predicted vs Actual

Die wichtigste Visualisierung für Regression - zeigt wie gut Predictions mit echten Werten übereinstimmen.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter Plot: Predicted vs Actual
axes[0].scatter(y, y_pred, alpha=0.3, s=10)
axes[0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Perfekte Vorhersage')
axes[0].set_xlabel('Tatsächliche Service Time (Minuten)', fontsize=12)
axes[0].set_ylabel('Vorhergesagte Service Time (Minuten)', fontsize=12)
axes[0].set_title('Predicted vs Actual Values', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Text mit Metriken
textstr = f'RMSE = {rmse:.2f}\nMAE = {mae:.2f}\nR² = {r2:.3f}'
axes[0].text(0.05, 0.95, textstr, transform=axes[0].transAxes, fontsize=11,
             verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Hexbin Plot (für dichte Bereiche besser sichtbar)
hexbin = axes[1].hexbin(y, y_pred, gridsize=50, cmap='YlOrRd', mincnt=1)
axes[1].plot([y.min(), y.max()], [y.min(), y.max()], 'b--', lw=2, label='Perfekte Vorhersage')
axes[1].set_xlabel('Tatsächliche Service Time (Minuten)', fontsize=12)
axes[1].set_ylabel('Vorhergesagte Service Time (Minuten)', fontsize=12)
axes[1].set_title('Predicted vs Actual (Density)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
plt.colorbar(hexbin, ax=axes[1], label='Anzahl Datenpunkte')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'model_predicted_vs_actual.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Predicted vs Actual Plot erstellt")

## 4. Visualisierung 2: Residuals (Fehler) Analyse

Residuals = Actual - Predicted. Zeigt systematische Fehler im Model.

In [ ]:
# Residuals berechnen
residuals = y - y_pred

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Residuals vs Predicted
axes[0, 0].scatter(y_pred, residuals, alpha=0.3, s=10)
axes[0, 0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0, 0].set_xlabel('Vorhergesagte Service Time (Minuten)', fontsize=11)
axes[0, 0].set_ylabel('Residuals (Actual - Predicted)', fontsize=11)
axes[0, 0].set_title('Residuals vs Predicted Values', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Residuals Histogram
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(x=0, color='r', linestyle='--', lw=2, label='Zero Error')
axes[0, 1].axvline(x=residuals.mean(), color='g', linestyle='--', lw=2, label=f'Mean: {residuals.mean():.2f}')
axes[0, 1].set_xlabel('Residuals (Minuten)', fontsize=11)
axes[0, 1].set_ylabel('Häufigkeit', fontsize=11)
axes[0, 1].set_title('Verteilung der Residuals', fontsize=13, fontweight='bold')
axes[0, 1].legend(fontsize=9)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Q-Q Plot (prüft Normalverteilung der Fehler)
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (Normalverteilung der Residuals)', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Absolute Errors
abs_errors = np.abs(residuals)
axes[1, 1].scatter(y_pred, abs_errors, alpha=0.3, s=10)
axes[1, 1].axhline(y=mae, color='r', linestyle='--', lw=2, label=f'MAE: {mae:.2f}')
axes[1, 1].set_xlabel('Vorhergesagte Service Time (Minuten)', fontsize=11)
axes[1, 1].set_ylabel('Absolute Errors (Minuten)', fontsize=11)
axes[1, 1].set_title('Absolute Errors vs Predicted', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=9)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'model_residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Residuals Analyse Plot erstellt")

## 5. Visualisierung 3: Error Distribution by Ranges

Zeigt ob das Model für bestimmte Service Time Bereiche besser/schlechter performt.

In [ ]:
# Service Time in Bereiche einteilen
y_series = pd.Series(y).reset_index(drop=True)
bins = [0, 5, 10, 15, 20, 25, 30, 100]
labels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30+']
time_ranges = pd.cut(y_series, bins=bins, labels=labels)

# DataFrame für Analyse
error_df = pd.DataFrame({
    'actual': y_series,
    'predicted': y_pred,
    'abs_error': np.abs(residuals),
    'time_range': time_ranges
})

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. MAE by Time Range
mae_by_range = error_df.groupby('time_range')['abs_error'].mean()
axes[0, 0].bar(range(len(mae_by_range)), mae_by_range.values, alpha=0.7, color='steelblue')
axes[0, 0].axhline(y=mae, color='r', linestyle='--', lw=2, label=f'Gesamt MAE: {mae:.2f}')
axes[0, 0].set_xticks(range(len(mae_by_range)))
axes[0, 0].set_xticklabels(mae_by_range.index, rotation=45)
axes[0, 0].set_xlabel('Service Time Range (Minuten)', fontsize=11)
axes[0, 0].set_ylabel('Mean Absolute Error (MAE)', fontsize=11)
axes[0, 0].set_title('MAE nach Service Time Bereichen', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=9)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Boxplot Errors by Range
error_df.boxplot(column='abs_error', by='time_range', ax=axes[0, 1])
axes[0, 1].set_xlabel('Service Time Range (Minuten)', fontsize=11)
axes[0, 1].set_ylabel('Absolute Error (Minuten)', fontsize=11)
axes[0, 1].set_title('Error Distribution nach Bereichen', fontsize=13, fontweight='bold')
axes[0, 1].get_figure().suptitle('')  # Remove default title
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)

# 3. Sample Count by Range
count_by_range = error_df.groupby('time_range').size()
axes[1, 0].bar(range(len(count_by_range)), count_by_range.values, alpha=0.7, color='lightcoral')
axes[1, 0].set_xticks(range(len(count_by_range)))
axes[1, 0].set_xticklabels(count_by_range.index, rotation=45)
axes[1, 0].set_xlabel('Service Time Range (Minuten)', fontsize=11)
axes[1, 0].set_ylabel('Anzahl Samples', fontsize=11)
axes[1, 0].set_title('Datenverteilung nach Bereichen', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. R² by Range (approximation)
r2_by_range = error_df.groupby('time_range').apply(
    lambda x: r2_score(x['actual'], x['predicted']) if len(x) > 1 else 0
)
axes[1, 1].bar(range(len(r2_by_range)), r2_by_range.values, alpha=0.7, color='mediumseagreen')
axes[1, 1].axhline(y=r2, color='r', linestyle='--', lw=2, label=f'Gesamt R²: {r2:.3f}')
axes[1, 1].set_xticks(range(len(r2_by_range)))
axes[1, 1].set_xticklabels(r2_by_range.index, rotation=45)
axes[1, 1].set_xlabel('Service Time Range (Minuten)', fontsize=11)
axes[1, 1].set_ylabel('R² Score', fontsize=11)
axes[1, 1].set_title('R² Score nach Bereichen', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=9)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'model_error_by_ranges.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Error Distribution by Ranges Plot erstellt")

## 6. Visualisierung 4: Feature Importance vs Error

Analysiert ob schlechte Predictions mit bestimmten Features zusammenhängen.

In [ ]:
# Errors zu Features hinzufügen
X_with_errors = X.copy()
X_with_errors['abs_error'] = np.abs(residuals)
X_with_errors['predicted'] = y_pred
X_with_errors['actual'] = y_series.values

# Top 6 wichtigste Features
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    top_features = feature_importance.head(6)['feature'].tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(top_features):
        axes[idx].scatter(X_with_errors[feature], X_with_errors['abs_error'], alpha=0.3, s=10)
        axes[idx].axhline(y=mae, color='r', linestyle='--', lw=1, alpha=0.5)
        axes[idx].set_xlabel(feature, fontsize=10)
        axes[idx].set_ylabel('Absolute Error (Min)', fontsize=10)
        axes[idx].set_title(f'{feature} vs Error', fontsize=11, fontweight='bold')
        axes[idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(PLOTS_DIR / 'model_features_vs_error.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Features vs Error Plot erstellt")
else:
    print("\n⚠ Model hat keine feature_importances_ (z.B. bei Linear Regression)")

## 7. Visualisierung 5: Error Percentiles

Zeigt die Verteilung der Errors in Perzentilen.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Cumulative Error Distribution
sorted_abs_errors = np.sort(np.abs(residuals))
cumulative = np.arange(1, len(sorted_abs_errors) + 1) / len(sorted_abs_errors)

axes[0].plot(sorted_abs_errors, cumulative * 100, linewidth=2)
axes[0].axvline(x=mae, color='r', linestyle='--', lw=2, label=f'MAE: {mae:.2f}')
axes[0].axhline(y=50, color='g', linestyle='--', lw=1, alpha=0.5, label='Median (50%)')
axes[0].axhline(y=90, color='orange', linestyle='--', lw=1, alpha=0.5, label='90% Percentile')
axes[0].set_xlabel('Absolute Error (Minuten)', fontsize=12)
axes[0].set_ylabel('Cumulative Percentage', fontsize=12)
axes[0].set_title('Kumulative Error Verteilung', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# 2. Error Percentiles Table
percentiles = [10, 25, 50, 75, 90, 95, 99]
error_percentiles = [np.percentile(np.abs(residuals), p) for p in percentiles]

axes[1].bar(range(len(percentiles)), error_percentiles, alpha=0.7, color='skyblue')
axes[1].set_xticks(range(len(percentiles)))
axes[1].set_xticklabels([f'{p}%' for p in percentiles])
axes[1].set_xlabel('Percentile', fontsize=12)
axes[1].set_ylabel('Absolute Error (Minuten)', fontsize=12)
axes[1].set_title('Error Percentiles', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Werte auf Balken anzeigen
for i, v in enumerate(error_percentiles):
    axes[1].text(i, v + 0.3, f'{v:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'model_error_percentiles.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Error Percentiles Plot erstellt")

# Percentiles ausgeben
print("\n" + "=" * 60)
print("ERROR PERCENTILES")
print("=" * 60)
for p, val in zip(percentiles, error_percentiles):
    print(f"{p:3d}% der Predictions haben einen Fehler <= {val:.2f} Minuten")

## 8. Zusammenfassung

Übersicht aller erstellten Visualisierungen und Erkenntnisse.

In [ ]:
print("=" * 80)
print("MODEL EVALUATION - ZUSAMMENFASSUNG")
print("=" * 80)

print("\n1. ERSTELLTE VISUALISIERUNGEN:")
print("   ✓ Predicted vs Actual (Scatter + Density)")
print("   ✓ Residuals Analyse (4 Plots)")
print("   ✓ Error Distribution by Time Ranges")
print("   ✓ Features vs Error Analysis")
print("   ✓ Error Percentiles & Cumulative Distribution")

print("\n2. PERFORMANCE METRIKEN:")
print(f"   - RMSE: {rmse:.4f} Minuten")
print(f"   - MAE:  {mae:.4f} Minuten")
print(f"   - R²:   {r2:.4f}")

print("\n3. ERROR VERTEILUNG:")
print(f"   - Median Error: {np.median(np.abs(residuals)):.2f} Minuten")
print(f"   - 90% der Fehler <= {np.percentile(np.abs(residuals), 90):.2f} Minuten")
print(f"   - Max Error: {np.max(np.abs(residuals)):.2f} Minuten")

print("\n4. INTERPRETATION:")
if r2 > 0.8:
    print("   ✓ EXCELLENT: Model erklärt > 80% der Varianz")
elif r2 > 0.6:
    print("   ✓ GOOD: Model erklärt > 60% der Varianz")
elif r2 > 0.4:
    print("   ⚠ FAIR: Model erklärt > 40% der Varianz")
else:
    print("   ❌ POOR: Model erklärt < 40% der Varianz")

print(f"\n   Im Durchschnitt weicht die Vorhersage um {mae:.2f} Minuten vom echten Wert ab.")

print("\n5. NÄCHSTE SCHRITTE:")
print("   - Residuals Analyse prüfen: Gibt es systematische Fehler?")
print("   - Q-Q Plot prüfen: Sind Fehler normalverteilt?")
print("   - Error by Ranges prüfen: Performt Model überall gleich gut?")
print("   - Features vs Error prüfen: Welche Features verursachen große Fehler?")

print("\n" + "=" * 80)
print("✓ EVALUATION ABGESCHLOSSEN")
print(f"✓ Alle Plots gespeichert in: {PLOTS_DIR}")
print("=" * 80)